# Data Processing script for the NSM/SWEML v2.0
This .ipynb script uses python module for retrieving NASA ASO observations, locating nearest SNOTEL sites, connecting SNOTEL obs with ASO obs, and add geospatial features to the ML training/testing/hindcast dataframes.

# Next steps 
- Revisist other scripts and convert to PyArrow/parquet Brocli compressed file storage
- replace row-by-roy for loops with lamba functions,  for aso_swe_file in tqdm(os.listdir(f"{TrainingDFpath}/Obsdf")):  #add file names to aso_swe_files
        aso_swe_files.append(aso_swe_file) list comprehension  == tiff_files = [filename for filename in os.listdir(folder_path) if filename.endswith(".tif")]
- VIIRS
- connect precip to DF,
- add new sites (e.g., regionval) to training DF with all the respective spatial resolution information
- connect regional data together to train model
- connect different regions
- add precipitation phase features (seasonal accumulated rain precip, seasonal accumulated snow precip as a function of temperature)
- explore adding other features stemming from SNOTEL, remote sensing (LULC), Snow Classifications (Sturms), energy balance
- add snotel script to functions

Put all units in SI, while it should not matter for model training since they are being normalized, they will be more interpretable.

In [2]:
region_list =    [ 'N_Sierras',
                                'Greater_Yellowstone',
                                'N_Co_Rockies',
                                'SW_Mont',
                                'SW_Co_Rockies',
                                'GBasin',
                                'N_Wasatch',
                                'N_Cascade',
                                'S_Wasatch',
                                'SW_Mtns',
                                'E_WA_N_Id_W_Mont',
                                'S_Wyoming',
                                'SE_Co_Rockies',
                                'Sawtooth',
                                'Ca_Coast',
                                'E_Or',
                                'N_Yellowstone',
                                'S_Cascade',
                                'Wa_Coast',
                                'Greater_Glacier']

In [5]:
from ASOget import ASODownload, ASODataProcessing

# Inputs for fetching ASO data for a region
short_name = 'ASO_50M_SWE'
version = '1'
time_start = '2013-04-02T00:00:00Z'
time_end = '2019-07-19T23:59:59Z'
region_list = ['S_Sierras']
output_res = 300 #desired spatial resoultion in meters (m)
directory = "Raw_ASO_Data"

#Get ASO data
for region in region_list:
    folder_name = f"{region}/{directory}"
    data_tool = ASODownload(short_name, version)
    b_box = data_tool.BoundingBox(region)  
    url_list = data_tool.cmr_search(time_start, time_end, region, b_box)
    data_tool.cmr_download(directory, region)

    #Convert ASO tifs to parquet
    data_processor = ASODataProcessing()
    data_processor.convert_tiff_to_parquet_multiprocess(folder_name, output_res, region) 

Bounding Box collected for S_Sierras: -120.3763448720203,36.29256774541929,-118.292253412863,38.994985247736324
Fetching file URLs in progress for S_Sierras from 2013-04-02T00:00:00Z to 2019-07-19T23:59:59Z
Querying for data:
	https://cmr.earthdata.nasa.gov/search/granules.json?provider=NSIDC_ECS&sort_key[]=start_date&sort_key[]=producer_granule_id&scroll=true&page_size=2000&short_name=ASO_50M_SWE&version=001&version=01&version=1&temporal[]=2013-04-02T00:00:00Z,2019-07-19T23:59:59Z&bounding_box=-120.3763448720203,36.29256774541929,-118.292253412863,38.994985247736324

Found 131 matches.
['https://n5eil01u.ecs.nsidc.org/DP1/ASO/ASO_50M_SWE.001/2013.04.03/ASO_50M_SWE_USCATB_20130403.tif', 'https://n5eil01u.ecs.nsidc.org/DP1/ASO/ASO_50M_SWE.001/2013.04.03/ASO_50M_SWE_USCATB_20130403.tif.xml', 'https://n5eil01u.ecs.nsidc.org/DP1/ASO/ASO_50M_SWE.001/2013.04.29/ASO_50M_SWE_USCATB_20130429.tif', 'https://n5eil01u.ecs.nsidc.org/DP1/ASO/ASO_50M_SWE.001/2013.04.29/ASO_50M_SWE_USCATB_20130429.tif

100%|██████████| 262/262 [00:00<00:00, 2632.90it/s]


  [============================================================] 100%  413.3kB/s   
  [============================================================] 100%  903.2kB/s   
  [============                                                ]  20%  4.0MB/s    [============                                                ]  20%  3.9MB/s    [========================                                    ]  40%  6.7MB/s    [========================                                    ]  40%  6.2MB/s    [====================================                        ]  60%  8.7MB/s   [================================================            ]  80%  10.8MB/s   [============================================================] 100%  13.2MB/s    [====================================                        ]  60%  8.2MB/s   
  [============                                                ]  20%  3.9MB/s    [================================================            ]  80%  9.8MB/s   [========================

100%|██████████| 131/131 [00:00<00:00, 184.59it/s]


An error occurred: '/home/rjohnson18/SWEMLv2.0/data/ASO/S_Sierras/Processed_300M_SWE/ASO_300M_20170717.tif' not recognized as a supported file format.
An error occurred: '/home/rjohnson18/SWEMLv2.0/data/ASO/S_Sierras/Processed_300M_SWE/ASO_300M_20180601.tif' not recognized as a supported file format.
An error occurred: '/home/rjohnson18/SWEMLv2.0/data/ASO/S_Sierras/Processed_300M_SWE/ASO_300M_20180426.tif' not recognized as a supported file format.
An error occurred: '/home/rjohnson18/SWEMLv2.0/data/ASO/S_Sierras/Processed_300M_SWE/ASO_300M_20180528.tif' not recognized as a supported file format.


...

 58%|█████▊    | 57/99 [00:00<00:00, 105.99it/s]

Corrupted thrift data at  2 :  12 13
Corrupted thrift data at  5 :  21 10
Corrupted thrift data at  12 :  40 10
Corrupted thrift data at  205 :  5 10
Corrupted thrift data at  227 :  6 15
Corrupted thrift data at  243 :  4 10
Corrupted thrift data at  330 :  22 10
Corrupted thrift data at  334 :  6 10
Corrupted thrift data at  355 :  9 10
Corrupted thrift data at  359 :  10 14
Corrupted thrift data at  360 :  17 0
Corrupted thrift data at  365 :  33 15
Corrupted thrift data at  374 :  6 10
Corrupted thrift data at  386 :  38 11
Corrupted thrift data at  392 :  65 11
Corrupted thrift data at  397 :  85 14
Corrupted thrift data at  466 :  -86 11
Corrupted thrift data at  467 :  -80 10
Corrupted thrift data at  468 :  -78 15
Corrupted thrift data at  469 :  -76 15
Corrupted thrift data at  470 :  -74 15
Corrupted thrift data at  615 :  90 13
Corrupted thrift data at  916 :  16 15
Corrupted thrift data at  918 :  6 15
Corrupted thrift data at  919 :  12 14
Corrupted thrift data at  922 :  

 69%|██████▊   | 68/99 [00:01<00:01, 29.47it/s] 

Attempt 2
Bad file conversion for ASO_300M_20150403.tif, attempting to reprocess


100%|██████████| 99/99 [00:01<00:00, 54.33it/s]


# Code for generating ML dataframe using nearest in situ monitoring sites

In [6]:
import GeoDF

# GeoDF used to create a dataframe for ML model development. Its function is to connect in situ observations to gridded locations
for region in region_list:
    #load snotel meta location data, use haversive function
    GeoDF.fetch_snotel_sites_for_cellids(region, output_res) # Using known up to date sites

    # Get geophysical attributes for each site, need to see how to add output resolution
    gdf = GeoDF.GeoSpatial(region, output_res)

    #use geodataframe with lat/long meta of all sites to determine slope, aspect, and elevation
    metadf = GeoDF.extract_terrain_data_threaded(gdf, region, output_res)




Loading all Geospatial prediction/observation files and concatenating into one dataframe


100%|██████████| 99/99 [00:03<00:00, 28.68it/s] 


Identifying unique sites to create geophysical information dataframe
converting to geodataframe
Processing snotel geometry
Calculating haversine distance for 299557 locations to in situ OBS, and saving cell-obs relationships in dictionary


  0%|          | 0/299557 [00:00<?, ?it/s]

Saving nearest SNOTEL in S_Sierras for each cell id in a pkl file
Loading geospatial data for S_Sierras
Converting to geodataframe
Calculating dataframe bounding box
Retrieving Copernicus 90m DEM tiles


...

There are 30 tiles in the region
Determining Grid Cell Spatial Features


100%|██████████| 299557/299557 [00:49<00:00, 6019.93it/s]


Job complete for getting geospatial metadata, processing dataframe


66590it [45:09,  6.60it/s]Exception ignored in: <function CachingFileManager.__del__ at 0x7f8d088b2d30>
Traceback (most recent call last):
  File "/home/rjohnson18/envs/SWEML_env2/lib/python3.9/site-packages/xarray/backends/file_manager.py", line 240, in __del__
    ref_count = self._ref_counter.decrement(self._key)
  File "/home/rjohnson18/envs/SWEML_env2/lib/python3.9/site-packages/xarray/backends/file_manager.py", line 293, in decrement
    count = self._counts[name] - 1
KeyError: [<function open at 0x7f8c8013c0d0>, ('https://elevationeuwest.blob.core.windows.net/copernicus-dem/COP90_hh/Copernicus_DSM_COG_30_N38_00_W120_00_DEM.tif?st=2024-05-02T03%3A19%3A57Z&se=2024-05-03T04%3A04%3A57Z&sp=rl&sv=2021-06-08&sr=c&skoid=c85c15d6-d1ae-42d4-af60-e2ca0f81359b&sktid=72f988bf-86f1-41af-91ab-2d7cd011db47&skt=2024-05-03T03%3A04%3A31Z&ske=2024-05-10T03%3A04%3A31Z&sks=b&skv=2021-06-08&sig=jZys/XTYcYnAOWj43EwTfhkdDmMCIkYHn/AQ2PK0Thk%3D',), 'r', (('sharing', False),)]
66595it [45:09,  9.38it/s]Exc

Saving S_Sierras dataframe in /home/rjohnson18/SWEMLv2.0/data/TrainingDFs/S_Sierras/300M_Resolution


## Connect Snotel to each ASO obs

In [7]:
import Obs_to_DF
region = "S_Sierras"
output_res = 300

#Connect nearest snotel observations with ASO data, makes a parquet file for each date  -  test to see if this works
dates = []
manual = False
Obs_to_DF.Nearest_Snotel_2_obs_MultiProcess(region, output_res, manual, dates) 


Connecting site observations with nearest monitoring network obs
Loading observations from 2013-2019
Loading 300M resolution grids for S_Sierras region
Processing datetime component of SNOTEL observation dataframe
Loading 99 processed ASO observations for the S_Sierras at 300M resolution
There are 99 aso dates in snotel obs
There are 0 missing snotel obs
Getting CDEC and SNOTEL observations for the following dates: []


0it [00:00, ?it/s]

Updating local meta and saving.


Connecting 99 timesteps of observations for S_Sierras


100%|██████████| 99/99 [00:11<00:00,  8.54it/s]


...

Site processing complete, adding observtional data to 20190715 df...


  0%|          | 0/390 [00:00<?, ?it/s]

Site processing complete, adding observtional data to 20190716 df...


  0%|          | 0/14404 [00:00<?, ?it/s]

Job complete for connecting SNOTEL obs to sites/dates


In [8]:
import GeoDF

region = 'S_Sierras'
output_res = 300

#Connect cell ids with ASO obs and snotel obs to geospatial features
GeoDF.add_geospatial_threaded(region, output_res)

Loading goeospatial meta data for grids in S_Sierras
Loading all available processed ASO observations for the S_Sierras at 300M resolution
Concatenating 99 with geospatial data...


  0%|          | 0/99 [00:00<?, ?it/s]

100%|██████████| 99/99 [00:11<00:00,  8.50it/s]


Job complete for connecting obs with geospatial data, the files can be found in /home/rjohnson18/SWEMLv2.0/data/TrainingDFs/S_Sierras/300M_Resolution/GeoObsDFs


# Get NASA VIIRS fraction snow covered area for each location 

In [9]:
import get_VIIRS_SCA
region = 'S_Sierras'
output_res = 300
threshold = 20

#check to see if the VIIRS data is available locally, if not, get from CIROH AWS
get_VIIRS_SCA.get_VIIRS_from_AWS()

#Connect VIIRS data to dataframes
get_VIIRS_SCA.augment_SCA_mutliprocessing(region, output_res, threshold)

Getting VIIRS fsca values for 99 timesteps of observations for S_Sierras
dataprocessing VIIRS for 20160614 complete...
dataprocessing VIIRS for 20160621 complete...dataprocessing VIIRS for 20190309 complete...

dataprocessing VIIRS for 20160607 complete...dataprocessing VIIRS for 20190715 complete...

dataprocessing VIIRS for 20150609 complete...
dataprocessing VIIRS for 20190611 complete...
dataprocessing VIIRS for 20190329 complete...
dataprocessing VIIRS for 20190609 complete...
dataprocessing VIIRS for 20190502 complete...
dataprocessing VIIRS for 20190714 complete...
dataprocessing VIIRS for 20190716 complete...
dataprocessing VIIRS for 20190604 complete...
dataprocessing VIIRS for 20190703 complete...
dataprocessing VIIRS for 20160426 complete...
dataprocessing VIIRS for 20130608 complete...
dataprocessing VIIRS for 20130601 complete...
dataprocessing VIIRS for 20190427 complete...
dataprocessing VIIRS for 20190326 complete...
dataprocessing VIIRS for 20190421 complete...
datapro

In [ ]:
#load access key
import pandas as pd
import numpy as np
import os
HOME = os.path.expanduser('~')
region = 'S_Sierras'
output_res = 300
threshold = 20

ViirsFolder = f"{HOME}/SWEMLv2.0/data/TrainingDFs/{region}/{output_res}M_Resolution/VIIRSGeoObsDFs/20_fSCA_Thresh"

#Get list of GeoObsDF dataframes
GeoObsDF_files = [filename for filename in os.listdir(ViirsFolder)]

file = pd.read_parquet(os.path.join(ViirsFolder, GeoObsDF_files[1]), engine = 'fastparquet')

file

In [ ]:
file[(file['VIIRS_SCA'].isna())]

In [ ]:
notna = file[(file['VIIRS_SCA']>0) & (file['swe_m'] > 0.5)]
np.sort(notna['VIIRS_SCA'].unique())

In [ ]:
np.NaN

In [ ]:
import get_Precip

#gets precipitation for each location, accumulates it through the water year

#set start/end date for a water year
years = [2013, 2014, 2015, 2016, 2017, 2018, 2019]
region = 'S_Sierras'
output_res = 100
for year in years:
    get_Precip.get_precip_threaded(year, region, output_res)

In [ ]:
import os
import pandas as pd

import pyarrow as pa
import pyarrow.parquet as pq

HOME = os.path.expanduser('~')

region = 'S_Sierras'
year = 2013
output_res = 100

Precippath = f"{HOME}/SWEMLv2.0/data/Precipitation/{region}/{output_res}M_NLDAS_Precip/{year}"

ppt = pd.read_parquet(f"{Precippath}/NLDAS_PPT_2013-04-03.parquet")

ppt.set_index('cell_id', inplace=True)

ppt

In [ ]:
ppt

In [ ]:
# Merge with metadata
req_cols = ['cell_id', 'lat', 'lon', 'BR_Coord_Long', 'BR_Coord_Lat', 'UR_Coord_Long', 'UR_Coord_Lat',
            'UL_Coord_Long', 'UL_Coord_Lat', 'BL_Coord_Long', 'BL_Coord_Lat', 'geometry']
Result = final_df.merge(metadata[req_cols], how='left', on='cell_id')

# Column renaming and ordering
Result.rename(columns={'swe': 'ASO_SWE_in'}, inplace=True)
Result = Result[['cell_id', 'Date', 'ASO_SWE_in', 'lat', 'lon', 'nearest site 1', 'nearest site 2',
                    'nearest site 3', 'nearest site 4', 'nearest site 5', 'nearest site 6',
                    'BR_Coord_Long', 'BR_Coord_Lat', 'UR_Coord_Long', 'UR_Coord_Lat',
                    'UL_Coord_Long', 'UL_Coord_Lat', 'BL_Coord_Long', 'BL_Coord_Lat']]

# Save the merged data to a new file
output_filename = f"{HOME}/SWEML/data/NSMv2.0/data/TrainingDFs/Merged_aso_snotel_data.parquet"
Result.to_csv(output_filename, index=False)
display(Result.head(10))
print("Processed and saved data")